# **Aplicación PLN para análisis y procesamiento de texto de carácter geopolítico, económico y político.**

**Primero de todo, extraeremos el corpus de documentos creando una aplicación para conectarnos a la API de Reddit y así poder extraer los datos.**

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import datetime as dt
import time

In [2]:
df = pd.read_csv('../../P3_Analisis_MorfoSintactico/data/t3_frases_ner.csv', encoding='utf-8', sep='"')
df

,texto
0,Juan PÃ©rez visitÃ³ a su abuela en el pueblo d...
1,"MarÃ­a GarcÃ­a, ganadora del premio de literat..."
2,"Carlos LÃ³pez, el nuevo director del proyecto,..."
3,"Ana MartÃ­nez, estudiante de medicina en la Un..."
4,"Luis FernÃ¡ndez, reconocido chef, inaugurÃ³ un..."
...,...
94,El cantante Pharrell Williams ofreciÃ³ un conc...
95,La ciudad de Toronto albergÃ³ una conferencia ...
96,El cientÃ­fico Louis Pasteur fue recordado en ...
97,La empresa Airbnb anunciÃ³ una nueva funciÃ³n ...
